# Serve Llama2-70b on SageMaker using the LMI container.


In this notebook, we deploy the [llama2-70b](https://huggingface.co/meta-llama/Llama-2-70b-hf) model on SageMaker by leveraging the [SageMaker Large Model Inference Container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers). For the purpose of this notebook, we'll use the weights from the following source:
- https://huggingface.co/TheBloke/Llama-2-70B-fp16


However, you can use the same approach to deploy the model using any other Llama2 weights like https://huggingface.co/meta-llama/Llama-2-70b-chat-hf, etc.

For information on llama2, please refer the paper [Llama 2: Open Foundation and Fine-Tuned Chat Models](https://arxiv.org/pdf/2307.09288.pdf).

This notebook explains how to deploy model optimized for latency and throughput. The tuning guide is available [LLM Tuning Guide](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers). There are some key Gen AI Patterns and use cases and they need different settings when hosting the model. The typical broad use case caterization would be 

1. Chatbot / QA. These applications need the ability to handle large model inputs and large model outputs. With contextual applications, these require prescriptive and factual responses back from LLM, which can be controlled by setting the appropriate decoding parameters. Latency and accuracy are top priorities.
2. Summarization. These applications usually have a large payload as input to the model and will have a small to medium output length payload. If we run these as batch we have some tolerence on latency, however throughput is a major concern. 
3. Generation. These applications usually have a smaller input payload but the open ended generation can involve generating to the full content length of the model. Throughput is of major concern.

# License agreement
View license information https://huggingface.co/meta-llama before using the model.
This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0.

## Install, import the required libraries; set some variables

In [ ]:
%pip install sagemaker boto3 huggingface_hub awscli --upgrade  --quiet

In [ ]:
import json
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

## Download the model artifacts and upload to S3
We recommend to first save the model in a S3 location and provide the S3 url in ENV variable `OPTION_MODEL_ID`. This allows faster downloads times.

In [ ]:
# Setup S3 bucket to download model from HuggingFace and store in S3
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_model_prefix = "llama-2-70B-fp16/lmi"  # folder within bucket where model artifact will go

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = "TheBloke/Llama-2-70B-fp16"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model", "*.safetensors"]

# Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

In [ ]:
# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

In [ ]:
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.model_id={model_artifact}")

### Select the appropriate configuration parameters and container

To optimize the deployment of Large Language Models (LLMs); one needs to choose the appropriate model partitioning framework, optimal batching technique, batching size, tensor parallelism degree, etc. The choice of a particular configuration depends on the use case.

Hence, based on the use case, you need to:
1. Set the configuration parameters for the container.
2. Select the appropriate container image to be used for inference.

## Use case: Open Ended generation - Chatbots, etc
Consider the following scenarios:
- Prompts with small input size and a small generated text
- Prompts with a small input size that generate a large number of tokens

Applications like chatbots, etc. have the above characteristics and also need to support high throughput. This needs to be taken into consideration while selecting the configuration parameters.

![chatbot](images/chatbot.png)


### Set the configuration parameters using environment variables
1. `SERVING_LOAD_MODELS` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the **Python** engine.

2. `OPTION_MODEL_ID`: Set this to the URI of the Amazon S3 bucket that contains the model. When this is set, the container leverages [s5cmd](https://github.com/peak/s5cmd) to download the model from s3. This enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance.
If you want to download the model from huggingface.co, you can set `OPTION_MODEL_ID` to the model id of a pre-trained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co.

3. `OPTION_TENSOR_PARALLEL_DEGREE`: Set to the number of accelerator devices over which DeepSpeed needs to partition the model. This parameter also controls the number of workers per model which will be started up when DJL serving runs. In this example we use the `ml.inf2.48xlarge` instance that has 12 accelerators; hence this is set to 12.

4. `OPTION_ROLLING_BATCH`: This parameter enables the use of a particular batching technique for continuous or iteration level batching to enable merging multiple concurrent requests that arrive at different times for inference.
In scenarios that involves open ended generation and chatbots, there is a need for having a high throughput. [vLLM](https://arxiv.org/pdf/2309.06180.pdf) is a fast LLM inference and serving framework that uses techniques like PagedAttention and continuous batching to improve the throughput. Hence, we set the `rolling_batch` parameter to `vllm`. When using `vllm`, you can also use some [additional parameters](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/configurations_large_model_inference_containers.md#vllm).

5. `OPTION_MAX_ROLLING_BATCH_SIZE`: The maximum number of concurrent requests to be used in a batch by the model server for inference. Clients can still send more requests to the endpoint, they will be queued.


For more information on the available options, please refer to the [DJL Serving - SageMaker Large Model Inference Configurations](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/configurations_large_model_inference_containers.md)


### Select the relevant Large Model Inference container
SageMaker offers optimized [large model inference containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers) that contains different frameworks for model parallelism enabling inference of LLMs on multiple accelerators.

In this scenario, since we are leveraging `vllm` as the batching technique, we leverage the `NeuronX` container that has frameworks like Transformers Neuronx, vllm, etc.

In [ ]:
image_uri = image_uris.retrieve(
    framework="djl-neuronx", 
    region=sess.boto_session.region_name, 
    version="0.26.0"
)

env_generation = {
    "HUGGINGFACE_HUB_CACHE": "/tmp",
    "TRANSFORMERS_CACHE": "/tmp",
    "SERVING_LOAD_MODELS": "test::Python=/opt/ml/model",
    "OPTION_MODEL_ID": model_artifact,
    "OPTION_TENSOR_PARALLEL_DEGREE": "24",  # Inf2.48xlarge has 12 Inferentia2 device (each with two NeuronCore-v2 cores).
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "8",
    "OPTION_MODEL_LOADING_TIMEOUT": "3600",
    "OPTION_ENTRYPOINT": "djl_python.transformers_neuronx",
    "OPTION_DTYPE": "fp16",
}

In [ ]:
print(image_uri)

In [ ]:
# Select the appropriate environment variable which will tune the deployment server.
env = env_generation  # use this in case it is 'generation' task

# Select the appropriate container
inference_image_uri = image_uri  # use this in case it is 'generation' task

print(f"Environment variables are ---- > {env}")
print(f"Image going to be used is ---- > {inference_image_uri}")

To create the end point the steps are:
- Create the Model using the inference image container

- Create the endpoint config using the following key parameters

In this notebook we leverage the boto3 SDK. You can also use the [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/).

### Create the Model
Leverage the `inference_image_uri` to create a model object. We will leverage the Least routing algorithim -- [Least Routing Algorithim](https://aws.amazon.com/blogs/machine-learning/minimize-real-time-inference-latency-by-using-amazon-sagemaker-routing-strategies/). This innovation from sagemnaker has shown to reduce latency by 10% or more when we have multiple instances configured to serve the endpoints

In [ ]:
model_name = sagemaker.utils.name_from_base("lmi-llama2-70b")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "Environment": env
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

### Create an endpoint config
Create an endpoint configuration using the appropriate instance type. Set the `ContainerStartupHealthCheckTimeoutInSeconds` to account for the time taken to download the LLM weights from S3 or the model hub; and the time taken to load the model on the Inferentia devices.

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.inf2.48xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
            "VolumeSizeInGB": 400
        },
    ],
)
endpoint_config_response

### Create an endpoint using the model and endpoint config

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### This step can take ~15 mins or longer

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Invoke the endpoint with a sample prompt

In [ ]:
# Use this for summarization use case test
prompt = """Briefly summarize this paragraph: Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases.
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition.
All of the Amazon Comprehend features accept UTF-8 text documents as the input. In addition, custom classification and custom entity recognition accept image files, PDF files, and Word files as input.
Amazon Comprehend can examine and analyze documents in a variety of languages, depending on the specific feature. For more information, see Languages supported in Amazon Comprehend. Amazon Comprehend’s Dominant language capability can examine documents and determine the dominant language for a far wider selection of languages."""
params = {"max_new_tokens": 64, "temperature": 0.1}

In [ ]:
# Use this for a chatbot, QA, or open-ended generation task use case test

prompt = "Amazon.com is the best"
params = {"max_new_tokens": 100, "do_sample": False}

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

## Clean up the environment

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

#### Resource:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [Deep Java Library for Large Model Inference](https://docs.djl.ai/docs/serving/serving/docs/large_model_inference.html)